# namedtuple - Tuple Subclass with Named Fields

The standard tuple uses numerical indexes to access its members.

In contrast, remembering which index should be used for each value can lead to errors, especially if the tuple has a lot of fields and is constructed far from where it is used. A namedtuple assigns names, as well as the numerical index, to each member.

In [1]:
# collections_tuple.py
bob = ('Bob', 30, 'male')
print('Representation:', bob)

jane = ('Jane', 29, 'female')
print('\nField by index:', jane[0])

print('\nFields by index:')
for p in [bob, jane]:
    print('{} is a {} year old {}'.format(*p))

Representation: ('Bob', 30, 'male')

Field by index: Jane

Fields by index:
Bob is a 30 year old male
Jane is a 29 year old female


## Defining

namedtuple instances are just as memory efficient as regular tuples because they do not have per-instance dictionaries. Each kind of namedtuple is represented by its own class, which is created by using the namedtuple() factory function. The arguments are the name of the new class and a string containing the names of the elements.

As the example illustrates, it is possible to access the fields of the namedtuple by name using dotted notation (obj.attr) as well as by using the positional indexes of standard tuples.

In [2]:
# collections_namedtuple_person.py
import collections

Person = collections.namedtuple('Person', 'name age')

bob = Person(name='Bob', age=30)
print('\nRepresentation:', bob)

jane = Person(name='Jane', age=29)
print('\nField by name:', jane.name)

print('\nFields by index:')
for p in [bob, jane]:
    print('{} is {} years old'.format(*p))


Representation: Person(name='Bob', age=30)

Field by name: Jane

Fields by index:
Bob is 30 years old
Jane is 29 years old


Just like a regular tuple, a namedtuple is immutable. This restriction allows tuple instances to have a consistent hash value, which makes it possible to use them as keys in dictionaries and to be included in sets.

Trying to change a value through its named attribute results in an AttributeError.

In [3]:
help(collections.namedtuple)

Help on function namedtuple in module collections:

namedtuple(typename, field_names, verbose=False, rename=False)
    Returns a new subclass of tuple with named fields.
    
    >>> Point = namedtuple('Point', ['x', 'y'])
    >>> Point.__doc__                   # docstring for the new class
    'Point(x, y)'
    >>> p = Point(11, y=22)             # instantiate with positional args or keywords
    >>> p[0] + p[1]                     # indexable like a plain tuple
    33
    >>> x, y = p                        # unpack like a regular tuple
    >>> x, y
    (11, 22)
    >>> p.x + p.y                       # fields also accessible by name
    33
    >>> d = p._asdict()                 # convert to a dictionary
    >>> d['x']
    11
    >>> Point(**d)                      # convert from a dictionary
    Point(x=11, y=22)
    >>> p._replace(x=100)               # _replace() is like str.replace() but targets named fields
    Point(x=100, y=22)



In [4]:
# collections_namedtuple_immutable.py

import collections

Person = collections.namedtuple('Person','name age')

pat = Person(name='Pat', age=12)

print('\nRepresentation: ', pat)

try:
    pat.age = 21
except Exception as e:
    print(e)


Representation:  Person(name='Pat', age=12)
can't set attribute


## Invalid Field Names

Field names are invalid if they are repeated or conflict with Python keywords.

As the field names are parsed, invalid values cause ValueError exceptions.

In [5]:
# collections_namedtuple_bad_fields.py

import collections

try:
    collections.namedtuple('Person', 'name class age')
except Exception as err:
    print(err)

try:
    collections.namedtuple('Person', 'name age age')
except Exception as err:
    print(err)

Type names and field names cannot be a keyword: 'class'
Encountered duplicate field name: 'age'


In situations where a namedtuple is created based on values outside the control of the program (such as to represent the rows returned by a database query, where the schema is not known in advance), the rename option should be set to True so the invalid fields are renamed.

The new names for renamed fields depend on their index in the tuple, so the field with name class becomes _1 and the duplicate age field is changed to _2.

In [6]:
# collections_namedtuple_rename.py

import collections

with_class = collections.namedtuple('Person', 'name class age', rename = True)
print(with_class._fields)

two_ages = collections.namedtuple('Person', 'name age age', rename = True)
print(two_ages._fields)

('name', '_1', 'age')
('name', 'age', '_2')


## Special Attributes

namedtuple provides several useful attributes and methods for working with subclasses and instances. All of these built-in properties have names prefixed with an underscore (_), which by convention in most Python programs indicates a private attribute. For namedtuple, however, the prefix is intended to protect the name from collision with user-provided attribute names.

The names of the fields passed to namedtuple to define the new class are saved in the _fields attribute.

Although the argument is a single space-separated string, the stored value is the sequence of individual names.

In [7]:
# collections_namedtuple_fields.py

import collections

Person = collections.namedtuple('Person','name age')

bob = Person(name = 'Bob', age = 30)
print(bob)
print(bob._fields)

Person(name='Bob', age=30)
('name', 'age')


namedtuple instances can be converted to OrderedDict instances using _asdict().

The keys of the OrderedDict are in the same order as the fields for the namedtuple.

In [8]:
# collections_namedtuple_asdict.py

import collections

Person = collections.namedtuple('Person','name age')

bob = Person(name = 'Bob', age = 30)
print(bob)
print(bob._asdict())

Person(name='Bob', age=30)
OrderedDict([('name', 'Bob'), ('age', 30)])


The _replace() method builds a new instance, replacing the values of some fields in the process.

Although the name implies it is modifying the existing object, because namedtuple instances are immutable the method actually returns a new object.

In [9]:
# collections_namedtuple_replace.py

import collections

Person = collections.namedtuple('Person', 'name age')

bob = Person(name = 'Bob', age = 30)
print(bob, id(bob))
bob2 = bob._replace(name='Alvin')
print(bob2, id(bob2))

print('Same?', bob is bob2)

Person(name='Bob', age=30) 4369047344
Person(name='Alvin', age=30) 4369147080
Same? False
